## Libraries

In [27]:
import pulp as lp
import random

## Src

In [28]:
def generate_vrptw_parameters(
    num_vehicles,
    num_customers,
    cost_range=(10, 50),
    time_range=(5, 30),
    demand_range=(5, 15),
    capacity=20,
    time_window_range=(0, 50),
    large_constant=1000,
):
    """
    Generates random VRPTW parameters for a given number of vehicles and customers.

    Parameters:
    - num_vehicles (int): Number of vehicles.
    - num_customers (int): Number of customers (not including depots).
    - cost_range (tuple): Range for travel costs between nodes.
    - time_range (tuple): Range for travel times between nodes.
    - demand_range (tuple): Range for customer demands.
    - capacity (int): Capacity of each vehicle.
    - time_window_range (tuple): Range for time windows (start and end times).
    - large_constant (int): Large constant for constraints.

    Returns:
    - dict: A dictionary containing all sets and parameters.
    """
    # Sets
    V = list(range(1, num_vehicles + 1))  # Vehicles labeled from 1 to num_vehicles
    N = list(
        range(num_customers + 2)
    )  # Nodes labeled from 0 to num_customers+1 (including depots)
    C = list(range(1, num_customers + 1))  # Customers labeled from 1 to num_customers

    # Parameters
    cij = {(i, j): random.randint(*cost_range) for i in N for j in N if i != j}
    tij = {(i, j): random.randint(*time_range) for i in N for j in N if i != j}
    di = {i: random.randint(*demand_range) for i in C}  # Customer demands
    q = capacity  # Vehicle capacity
    a = {i: random.randint(*time_window_range) for i in N}  # Start of time window
    b = {
        i: random.randint(a[i] + 1, time_window_range[1]) for i in N
    }  # End of time window, after start time
    K = large_constant  # Large constant for time window constraints

    # Ensuring depot time windows
    a[0], b[0] = 0, time_window_range[1]  # Starting depot time window
    a[num_customers + 1], b[num_customers + 1] = (
        0,
        time_window_range[1],
    )  # Returning depot time window

    return {
        "V": V,
        "N": N,
        "C": C,
        "cij": cij,
        "tij": tij,
        "di": di,
        "q": q,
        "a": a,
        "b": b,
        "K": K,
    }

## Constants

In [29]:
n = 2  # num of customers
num_vehicles = 2
parameters = generate_vrptw_parameters(num_vehicles=num_vehicles, num_customers=n)
print(parameters)
V, N, C, cij, tij, di, q, a, b, K = parameters.values()

{'V': [1, 2], 'N': [0, 1, 2, 3], 'C': [1, 2], 'cij': {(0, 1): 25, (0, 2): 45, (0, 3): 35, (1, 0): 33, (1, 2): 48, (1, 3): 30, (2, 0): 42, (2, 1): 47, (2, 3): 30, (3, 0): 36, (3, 1): 16, (3, 2): 39}, 'tij': {(0, 1): 6, (0, 2): 15, (0, 3): 24, (1, 0): 26, (1, 2): 18, (1, 3): 19, (2, 0): 5, (2, 1): 9, (2, 3): 10, (3, 0): 20, (3, 1): 12, (3, 2): 16}, 'di': {1: 13, 2: 12}, 'q': 20, 'a': {0: 0, 1: 23, 2: 26, 3: 0}, 'b': {0: 50, 1: 27, 2: 35, 3: 50}, 'K': 1000}


In [30]:
# for i in N:
#     a[i], b[i] = 0, 100

In [31]:
print(a)
print(b)

{0: 0, 1: 23, 2: 26, 3: 0}
{0: 50, 1: 27, 2: 35, 3: 50}


Modèle de minimisation de coût

In [32]:
problem = lp.LpProblem("VRPTW", lp.LpMinimize)

Dicision variables

In [33]:
x = lp.LpVariable.dicts("x", ((k, i, j) for k in V for i in N for j in N if i != j), cat='Binary')
s = lp.LpVariable.dicts("s", ((i, k) for i in N for k in V), lowBound=0, cat='Continuous')

Objective function

In [34]:
problem += lp.lpSum(cij[i, j] * x[k, i, j] for k in V for i in N for j in N if i != j)

## Constraints

Each customer is visited exactly once

In [35]:
for i in C:
    problem += lp.lpSum(x[k, i, j] for k in V for j in N if i != j) == 1


Vehicle capacity constraint

In [36]:

for k in V:
    problem += lp.lpSum(di[i] * x[k, i, j] for i in C for j in N if i != j) <= q


Each vehicle starts at the depot (node 0)

In [37]:
for k in V:
    problem += lp.lpSum(x[k, 0, j] for j in N if j != 0) == 1

Flow conservation for each customer

In [38]:
for k in V:
    for h in C:
        problem += lp.lpSum(x[k, i, h] for i in N if i != h) == lp.lpSum(
            x[k, h, j] for j in N if h != j
        )


Each vehicle ends at the returning depot (node n+1)

In [39]:
for k in V:
    problem += lp.lpSum(x[k, i, n + 1] for i in N if i != n + 1) == 1

In [40]:
problem

VRPTW:
MINIMIZE
25*x_(1,_0,_1) + 45*x_(1,_0,_2) + 35*x_(1,_0,_3) + 33*x_(1,_1,_0) + 48*x_(1,_1,_2) + 30*x_(1,_1,_3) + 42*x_(1,_2,_0) + 47*x_(1,_2,_1) + 30*x_(1,_2,_3) + 36*x_(1,_3,_0) + 16*x_(1,_3,_1) + 39*x_(1,_3,_2) + 25*x_(2,_0,_1) + 45*x_(2,_0,_2) + 35*x_(2,_0,_3) + 33*x_(2,_1,_0) + 48*x_(2,_1,_2) + 30*x_(2,_1,_3) + 42*x_(2,_2,_0) + 47*x_(2,_2,_1) + 30*x_(2,_2,_3) + 36*x_(2,_3,_0) + 16*x_(2,_3,_1) + 39*x_(2,_3,_2) + 0
SUBJECT TO
_C1: x_(1,_1,_0) + x_(1,_1,_2) + x_(1,_1,_3) + x_(2,_1,_0) + x_(2,_1,_2)
 + x_(2,_1,_3) = 1

_C2: x_(1,_2,_0) + x_(1,_2,_1) + x_(1,_2,_3) + x_(2,_2,_0) + x_(2,_2,_1)
 + x_(2,_2,_3) = 1

_C3: 13 x_(1,_1,_0) + 13 x_(1,_1,_2) + 13 x_(1,_1,_3) + 12 x_(1,_2,_0)
 + 12 x_(1,_2,_1) + 12 x_(1,_2,_3) <= 20

_C4: 13 x_(2,_1,_0) + 13 x_(2,_1,_2) + 13 x_(2,_1,_3) + 12 x_(2,_2,_0)
 + 12 x_(2,_2,_1) + 12 x_(2,_2,_3) <= 20

_C5: x_(1,_0,_1) + x_(1,_0,_2) + x_(1,_0,_3) = 1

_C6: x_(2,_0,_1) + x_(2,_0,_2) + x_(2,_0,_3) = 1

_C7: x_(1,_0,_1) - x_(1,_1,_0) - x_(1,_1,_2) - x_(1

Time window constraint with large constant K

In [41]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += s[i, k] + tij[i, j] - K * (1 - x[k, i, j]) <= s[j, k]

Start time must be within the time window

In [42]:
for i in N:
    for k in V:
        problem += a[i] <= s[i, k]
        problem += s[i, k] <= b[i]


Binary constraint for x variables

In [43]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += x[k, i, j] >= 0
                problem += x[k, i, j] <= 1

## Solution

In [44]:
problem.solve()

for var in problem.variables():
    print(f"{var.name} = {var.varValue}")

print("Solution Status:", lp.LpStatus[problem.status])
print("Objective value:", lp.value(problem.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cuphead/Projects/VRPTW-Meta-Heuristique/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/435632936cdd48c7bd811bd61fa48622-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/435632936cdd48c7bd811bd61fa48622-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 105 COLUMNS
At line 374 RHS
At line 475 BOUNDS
At line 500 ENDATA
Problem MODEL has 100 rows, 32 columns and 196 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 130 - 0.00 seconds
Cgl0003I 10 fixed, 0 tightened bounds, 20 strengthened rows, 31 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 23 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed

Get the routes for each vehicle

In [45]:
# sol_var = {}
# for k in V:
#     for i in N:
#         for j in N:
#             if i!=j:
#                 sol_var[(k, i, j)] = lp.value(x[k, i, j])

In [51]:
routes = {k: [] for k in V} 

for k in V:
    current_node = 0  # Start from the depot (node 0)
    while (
        current_node != n + 1
    ):  # Continue until reaching the returning depot (node n+1)
        for j in N:
            #if current_node != j and sol_var[k, current_node, j] == 1:
            if current_node != j and lp.value(x[k, current_node, j]) == 1:
                print(f"j:{j}, k:{k}, current node:{current_node}")
                routes[k].append((current_node, j))
                current_node = j
                break

j:2, k:1, current node:0
j:3, k:1, current node:2
j:1, k:2, current node:0
j:3, k:2, current node:1


Print the routes

In [47]:
for k in V:
    route_str = " -> ".join(str(node) for node, _ in routes[k]) + f" -> {n + 1}"
    print(f"Route for vehicle {k}: {route_str}")

Route for vehicle 1: 0 -> 2 -> 3
Route for vehicle 2: 0 -> 1 -> 3


**Note:** is 0 (n+1 and 0 are bothe the depot)

Display service start times for each customer

In [48]:
for k in V:
    print(f"\nService start times for vehicle {k}:")
    for i in N:
        start_time = lp.value(s[i, k])
        if (
            start_time is not None and start_time > 0
        ):  # Only print if the vehicle visits the customer
            print(f"  Customer {i}: Start time = {start_time:.2f}")


Service start times for vehicle 1:
  Customer 1: Start time = 23.00
  Customer 2: Start time = 26.00
  Customer 3: Start time = 50.00

Service start times for vehicle 2:
  Customer 1: Start time = 23.00
  Customer 2: Start time = 26.00
  Customer 3: Start time = 50.00
